In [14]:
import pandas as pd
import numpy as np
# from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, roc_auc_score
from scipy.stats import iqr
from sklearn.decomposition import PCA
from IPython.display import display, HTML
# import optuna.integration.lightgbm as lgb
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
import warnings
warnings. filterwarnings('ignore')

### Part 1: Predict drug response on a per drug basis across the GDSC cell lines {-}

Load files that contain gene expression information, fitted dose response results, and the information of the screen drugs.

In [3]:
# load and process dataset for gene expression levels on a cell-line basis
cell_line_rma = pd.read_csv('Cell_line_RMA_proc_basalExp_transposed.tsv', delimiter='\t')
cell_line_rma_dscrpt = cell_line_rma.iloc[0,:].copy()
cell_line_rma_data = cell_line_rma.iloc[1:,:].copy()
cell_line_rma_data['GENE_SYMBOLS'] = cell_line_rma_data['GENE_SYMBOLS'].str.split(".",expand=True)[1]
cell_line_rma_data_unique = cell_line_rma_data.drop_duplicates(subset=['GENE_SYMBOLS']).apply(pd.to_numeric)

# load dataset for cell-line and drug combination IC50 values
fitted_dose_response = pd.read_csv('v17_fitted_dose_response.tsv', delimiter='\t')

# load the file with drug names and information
GDSC_screened_compounds = pd.read_csv('GDSC_Screened_Compounds.tsv', delimiter='\t')

#### Details on model training

The IC50 values were transformed into three categories (sensitive, intermediate, resistant) using the first and third quantiles of IC50 for each drug as threshold values.

Stratified 5-fold cross-validation was employed for model development and evaluation, keeping class prevalence constant in each fold.

PCA was used to reduce the feature dimension. I experimented with various numbers of components and determined that 50 would provide the highest efficiency without sacrificing model performance.

Light Gradient Boosted Machine (LightGBM), a gradient boosting framework that uses tree based learning algorithms designed to have high efficiency, was used to make the predictions.
A LightGBM regression model and a LightGBM classification model were trained on the same cross-validation split.

Due to limitations on computational power, I'm not including the hyperparameter tuning step here, but it was performed separately and the parameters used here were selected from that process.
Class weights were set to be inversely proportional to the class prevalence in order to address the class imbalance issue.
Learning rate was reduced over time as a function of the current iteration number.
Early stopping was implemented to avoid overfitting and reduce training time.

RMSE was calculated for the regression model. Accuracy, F1-score, and AUC were calculated for the classification model. The F1-score calculated in each fold was the average F1-score of the three classes, weighted by the number of true instances for each label. The AUC calculated in each fold was the average AUC of all possible pairwise combinations of classes, weighted by the number of true instances for each label. For each drug, the metrics were first recorded for each of the 5 cross-validation folds, and finally I reported the metrics averaged across the 5 folds.

In [ ]:
drug_ids = pd.unique(fitted_dose_response.DRUG_ID)
n_splits = 5
rmse = np.zeros((len(drug_ids),n_splits))
accuracy = np.zeros((len(drug_ids),n_splits))
f1 = np.zeros((len(drug_ids),n_splits))
auc = np.zeros((len(drug_ids),n_splits))

for i, drug in enumerate(drug_ids):
    fitted_dose_response_oneDrug = fitted_dose_response[fitted_dose_response['DRUG_ID']==drug]
    df = pd.merge(fitted_dose_response_oneDrug, cell_line_rma_data_unique, how="inner", left_on='COSMIC_ID', right_on='GENE_SYMBOLS')
    X = df.iloc[:,9:].copy()
    y = df['LN_IC50'].copy()
    
    # determine IC50 threshold values for SIR: 1st and 3rd quartile
    cutoff1, cutoff2 = y.quantile([0.25, 0.75])
    
    # categorical truth, encoded into numerical levels
    y_label = pd.cut(y, [-np.inf,cutoff1,cutoff2,np.inf], labels=['sensitive','intermediate','resistant'])
    le = LabelEncoder()
    le.fit(y_label)
    y_le = le.transform(y_label)

    # train and evaluate using stratified 5-fold cross-validation
    for j, (train, test) in enumerate(StratifiedKFold(n_splits=n_splits, random_state=7).split(X, y_label)):
        # feature selection/dimension reduction
        pca = PCA(n_components=50).fit(X.iloc[train])
        X_train = pca.transform(X.iloc[train])
        X_test = pca.transform(X.iloc[test])

        # regression model
        # splitting training fold into training and validation sets
        train_x, val_x, train_y, val_y = train_test_split(X_train, y[train], test_size=0.2)
        dtrain = lgb.Dataset(train_x, label=train_y)
        dval = lgb.Dataset(val_x, label=val_y)

        params = {"objective": "regression",
                  "metric": "l2",
                  "verbose": -1,
                  "boosting_type": "gbdt",
                  "class_weight": "balanced",
                  "num_leaves": 31,
                  "max_depth": 10,
                  "min_child_samples": 20,
                  "feature_fraction": 1}

        model_rg = lgb.train(params, dtrain, valid_sets=[dtrain, dval], 
                             verbose_eval=100, early_stopping_rounds=50,
                             learning_rates = lambda iter: 0.05 * (0.99 ** iter))
        
        y_pred_rg = model_rg.predict(X_test, num_iteration=model_rg.best_iteration)
        y_true = y[test]
        rmse[i,j] = mean_squared_error(y_true, y_pred_rg, squared=False)
        
        # classification model (SIR)
        train_x, val_x, train_y, val_y = train_test_split(X_train, y_le[train], test_size=0.2)
        dtrain = lgb.Dataset(train_x, label=train_y, params={'verbose': -1})
        dval = lgb.Dataset(val_x, label=val_y, params={'verbose': -1})

        params = {"objective": "multiclass",
                  "num_class": 3,
                  "metric": "multi_logloss",
                  "verbose": -1,
                  "boosting_type": "gbdt",
                  "class_weight": "balanced",
                  "num_leaves": 31,
                  "max_depth": 10,
                  "min_child_samples": 20,
                  "feature_fraction": 1}

        model_cl = lgb.train(params, dtrain, valid_sets=[dtrain, dval],
                             verbose_eval=100, early_stopping_rounds=50,
                             learning_rates = lambda iter: 0.05 * (0.99 ** iter))
        
        y_pred_cl = model_cl.predict(X_test, num_iteration=model_cl.best_iteration)
        y_pred_cl_cat = np.argmax(y_pred_cl, axis=1)
        y_true_cat = y_le[test]
        
        accuracy[i,j] = accuracy_score(y_true_cat, y_pred_cl_cat)
        # average F1-score of the three labels, weighted by the number of true instances for each label
        f1[i,j] = f1_score(y_true_cat, y_pred_cl_cat, average='weighted')
        # average AUC of all possible pairwise combinations of classes, weighted by the number of true instances for each label
        auc[i,j] = roc_auc_score(y_true_cat, y_pred_cl, multi_class='ovo', average='weighted')

# average evaluation metrics over the 5 folds, summarize in a dataframe, and sort by RMSE
scores1 = pd.DataFrame(np.c_[drug_ids, np.mean(rmse, axis=1), np.mean(accuracy, axis=1), np.mean(f1, axis=1), np.mean(auc, axis=1)],
                   columns=['Drug_ID','RMSE','Accuracy','F1-score','AUC']).sort_values('RMSE')
scores1['Drug_ID'] = scores1['Drug_ID'].astype('int64')

#### Results

The top 10 and bottom 10 results (ranked by RMSE) are displayed below, including the drug ID and drug name along with all the performance metrics. Note that the performance ranked by RMSE is not necessarily the same as the ranking according to other metrics.

In [20]:
# merge scores with the file that contains drug names by drug ID
scores1 = pd.merge(scores1, GDSC_screened_compounds, left_on="Drug_ID", right_on="DRUG ID").iloc[:,np.r_[0,6,1:5]]
# show the top and bottom 10 results as ranked by RMSE
print('Top 10 results:')
display(scores1.head(10))
print('Bottom 10 results:')
display(scores1.tail(10))

Top 10 results:


,Drug_ID,DRUG NAME,RMSE,Accuracy,F1-score,AUC
0,1262,UNC1215,0.467885,0.563934,0.544210,0.694333
1,266,Zibotentan,0.481658,0.553737,0.508785,0.687672
2,150,Bicalutamide,0.496250,0.512013,0.461134,0.626315
3,1264,SGC0946,0.536468,0.540503,0.481954,0.652593
4,341,EX-527,0.584964,0.510899,0.469021,0.659880
5,91,KIN001-135,0.603429,0.503549,0.441034,0.618908
6,205,BMS-708163,0.609634,0.487532,0.447266,0.627374
7,1502,Bicalutamide,0.619808,0.495487,0.394423,0.580183
8,1018,Veliparib,0.669887,0.510595,0.401222,0.602204
9,312,Tivozanib,0.676975,0.560253,0.531834,0.683065


Bottom 10 results:


,Drug_ID,DRUG NAME,RMSE,Accuracy,F1-score,AUC
255,299,OSI-027,2.051096,0.558375,0.522198,0.682304
256,344,THZ-2-49,2.077956,0.494553,0.443527,0.599821
257,302,PI-103,2.085635,0.563442,0.531582,0.678098
258,51,Dasatinib,2.106321,0.548038,0.527825,0.684081
259,3,Rapamycin,2.111996,0.487519,0.339373,0.540789
260,346,THZ-2-102-1,2.208040,0.564745,0.525582,0.687349
261,190,Bleomycin,2.273888,0.490751,0.373401,0.582560
262,268,YM155,2.308411,0.503929,0.394952,0.594042
263,1248,FK866,2.605698,0.598895,0.563300,0.719840
264,135,Gemcitabine,2.628593,0.468248,0.386788,0.570268


### Part 2: Include additional assay types to improve model in part 1 {-}

Load files that contain the RACS results for cell lines.

In [21]:
# load and process dataset for RACS in cell lines
RACS_in_cell_lines = pd.read_csv('RACS_in_cell_lines.tsv', delimiter='\t')
# when multiple readings exist for a cell line, keep the first one
RACS_in_cell_lines_unique = RACS_in_cell_lines.drop_duplicates(subset=['COSMIC_ID'])
# simplify 'Region identifier' by removing anything in the parentheses - did not affect number of levels
RACS_in_cell_lines_unique['Region identifier short'] = RACS_in_cell_lines_unique['Region identifier'].str.split(" ", expand=True)[0]
# transform three potentially useful categorical variables to one-hot encoding
RACS_in_cell_lines_unique_onehot = pd.get_dummies(RACS_in_cell_lines_unique, columns=['Cancer Type','Alteration Type','Region identifier short'])

#### Details on model training

I included 3 additional categorical features from the RACS assay results: cancer type, alteration type, and region identifier. These 3 features contributed to an additional 204 total featuers after one-hot encoding.

The model training process followed that of part 1. Based on experimentation, the optimal number of components to keep during PCA was still 50.

In [ ]:
rmse = np.zeros((len(drug_ids),n_splits))
accuracy = np.zeros((len(drug_ids),n_splits))
f1 = np.zeros((len(drug_ids),n_splits))
auc = np.zeros((len(drug_ids),n_splits))

for i, drug in enumerate(drug_ids):
    fitted_dose_response_oneDrug = fitted_dose_response[fitted_dose_response['DRUG_ID']==drug]
    df = pd.merge(fitted_dose_response_oneDrug, cell_line_rma_data_unique, how="inner", left_on='COSMIC_ID', right_on='GENE_SYMBOLS')
    df = pd.merge(df, RACS_in_cell_lines_unique_onehot.iloc[:,np.r_[1, 5:RACS_in_cell_lines_unique_onehot.shape[1]]], how="inner", on='COSMIC_ID')
    X = df.iloc[:,9:].copy()
    y = df['LN_IC50'].copy()
    
    # determine IC50 threshold values for SIR: 1st and 3rd quartile
    cutoff1, cutoff2 = y.quantile([0.25, 0.75])
    
    # categorical truth, encoded into numerical levels
    y_label = pd.cut(y, [-np.inf,cutoff1,cutoff2,np.inf], labels=['sensitive','intermediate','resistant'])
    le = LabelEncoder()
    le.fit(y_label)
    y_le = le.transform(y_label)

    # train and evaluate using stratified 5-fold cross-validation
    for j, (train, test) in enumerate(StratifiedKFold(n_splits=n_splits, random_state=7).split(X, y_label)):
        # feature selection/dimension reduction
        pca = PCA(n_components=50).fit(X.iloc[train])
        X_train = pca.transform(X.iloc[train])
        X_test = pca.transform(X.iloc[test])

        # regression model
        # splitting training fold into training and validation sets
        train_x, val_x, train_y, val_y = train_test_split(X_train, y[train], test_size=0.2)
        dtrain = lgb.Dataset(train_x, label=train_y)
        dval = lgb.Dataset(val_x, label=val_y)

        params = {"objective": "regression",
                  "metric": "l2",
                  "verbose": -1,
                  "boosting_type": "gbdt",
                  "class_weight": "balanced",
                  "num_leaves": 31,
                  "max_depth": 10,
                  "min_child_samples": 20,
                  "feature_fraction": 1}

        model_rg = lgb.train(params, dtrain, valid_sets=[dtrain, dval], 
                             verbose_eval=100, early_stopping_rounds=50,
                             learning_rates = lambda iter: 0.05 * (0.99 ** iter))
        
        y_pred_rg = model_rg.predict(X_test, num_iteration=model_rg.best_iteration)
        y_true = y[test]
        rmse[i,j] = mean_squared_error(y_true, y_pred_rg, squared=False)
        
        # classification model (SIR)
        train_x, val_x, train_y, val_y = train_test_split(X_train, y_le[train], test_size=0.2)
        dtrain = lgb.Dataset(train_x, label=train_y, params={'verbose': -1})
        dval = lgb.Dataset(val_x, label=val_y, params={'verbose': -1})

        params = {"objective": "multiclass",
                  "num_class": 3,
                  "metric": "multi_logloss",
                  "verbose": -1,
                  "boosting_type": "gbdt",
                  "class_weight": "balanced",
                  "num_leaves": 31,
                  "max_depth": 10,
                  "min_child_samples": 20,
                  "feature_fraction": 1}

        model_cl = lgb.train(params, dtrain, valid_sets=[dtrain, dval],
                             verbose_eval=100, early_stopping_rounds=50,
                             learning_rates = lambda iter: 0.05 * (0.99 ** iter))
        
        y_pred_cl = model_cl.predict(X_test, num_iteration=model_cl.best_iteration)
        y_pred_cl_cat = np.argmax(y_pred_cl, axis=1)
        y_true_cat = y_le[test]
        
        accuracy[i,j] = accuracy_score(y_true_cat, y_pred_cl_cat)
        # average F1-score of the three labels, weighted by the number of true instances for each label
        f1[i,j] = f1_score(y_true_cat, y_pred_cl_cat, average='weighted')
        # average AUC of all possible pairwise combinations of classes, weighted by the number of true instances for each label
        auc[i,j] = roc_auc_score(y_true_cat, y_pred_cl, multi_class='ovo', average='weighted')

# average evaluation metrics over the 5 folds, summarize in a dataframe, and sort by RMSE
scores2 = pd.DataFrame(np.c_[drug_ids, np.mean(rmse, axis=1), np.mean(accuracy, axis=1), np.mean(f1, axis=1), np.mean(auc, axis=1)],
                   columns=['Drug_ID','RMSE','Accuracy','F1-score','AUC']).sort_values('RMSE')
scores2['Drug_ID'] = scores2['Drug_ID'].astype('int64')

#### Results and interpretation

From the top 10 and bottom 10 results displayed, the results seem comparable to those in part 1.

Although additional features were included to enrich available information for each cell line, the number of samples was reduced becuase the RACS assay results were not available for all cell lines. The performance may be furhter improved if more additional features are included and are available for most samples.

In [26]:
# merge scores with the file that contains drug names by drug ID
scores2 = pd.merge(scores2, GDSC_screened_compounds, left_on="Drug_ID", right_on="DRUG ID").iloc[:,np.r_[0,6,1:5]]
# show the top and bottom 10 results as ranked by RMSE
print('Top 10 results:')
display(scores2.head(10))
print('Bottom 10 results:')
display(scores2.tail(10))

Top 10 results:


,Drug_ID,DRUG NAME,RMSE,Accuracy,F1-score,AUC
0,1262,UNC1215,0.472518,0.508163,0.416980,0.612979
1,266,Zibotentan,0.488698,0.470889,0.351787,0.538805
2,150,Bicalutamide,0.513033,0.501099,0.378952,0.545406
3,1264,SGC0946,0.550619,0.482558,0.343213,0.538195
4,312,Tivozanib,0.558262,0.508990,0.385626,0.562801
5,91,KIN001-135,0.576615,0.500000,0.355595,0.488576
6,205,BMS-708163,0.579681,0.494909,0.373588,0.535001
7,341,EX-527,0.590254,0.490909,0.347217,0.595839
8,1502,Bicalutamide,0.615387,0.481881,0.345217,0.506813
9,1375,Temozolomide,0.639171,0.513318,0.424858,0.611261


Bottom 10 results:


,Drug_ID,DRUG NAME,RMSE,Accuracy,F1-score,AUC
255,302,PI-103,2.124116,0.507071,0.369398,0.570876
256,299,OSI-027,2.126953,0.481818,0.345569,0.533079
257,344,THZ-2-49,2.140791,0.498990,0.353219,0.545002
258,56,WH-4-023,2.251761,0.484308,0.330523,0.536695
259,51,Dasatinib,2.287476,0.515385,0.377057,0.584219
260,190,Bleomycin,2.319254,0.496703,0.338232,0.494091
261,346,THZ-2-102-1,2.334517,0.503051,0.367316,0.560970
262,268,YM155,2.380884,0.494737,0.356004,0.522835
263,1248,FK866,2.509282,0.508333,0.418777,0.601301
264,135,Gemcitabine,2.776634,0.497802,0.361919,0.555571


### Part 3: Rank drugs for given cell line {-}

Because there were $265 \times 5 = 1325$ different by-drug regression models from the previous questions, I chose not to save them. In this part I retrained them in the same way to make predictions in this question. The same 5-fold cross-validation scheme was used as before, training the model on 80% of the data and testing on the rest 20% in each iteration. This time, rather than calculating evaluation metrics on the test sets, all predictions were saved to perform ranking in the end when we have scores for all drugs and all cell lines.

In [ ]:
# dataframes for IC_50 results from model prediction and experiments
scores = pd.DataFrame(columns = drug_ids, index = cell_line_rma_data_unique.GENE_SYMBOLS) 
exprmt = pd.DataFrame(columns = drug_ids, index = cell_line_rma_data_unique.GENE_SYMBOLS) 

for i, drug in enumerate(drug_ids):
    fitted_dose_response_oneDrug = fitted_dose_response[fitted_dose_response['DRUG_ID']==drug]
    df = pd.merge(fitted_dose_response_oneDrug, cell_line_rma_data_unique, how="inner", left_on='COSMIC_ID', right_on='GENE_SYMBOLS')
    X = df.iloc[:,9:].copy()
    y = df['LN_IC50'].copy()
    ids = df['COSMIC_ID'].copy()
    
    # add experimental results for this drug to a dataframe
    exprmt.loc[df.GENE_SYMBOLS, drug] = y.values
    
    # determine IC50 threshold values for SIR: 1st and 3rd quartile
    cutoff1, cutoff2 = y.quantile([0.25, 0.75])
    
    # categorical truth, encoded into numerical levels
    y_label = pd.cut(y, [-np.inf,cutoff1,cutoff2,np.inf], labels=['sensitive','intermediate','resistant'])

    # train and predict using stratified 5-fold cross-validation
    for j, (train, test) in enumerate(StratifiedKFold(n_splits=n_splits, random_state=7).split(X, y_label)):
        # feature selection/dimension reduction
        pca = PCA(n_components=50).fit(X.iloc[train])
        X_train = pca.transform(X.iloc[train])
        X_test = pca.transform(X.iloc[test])

        # regression model
        # splitting training fold into training and validation sets
        train_x, val_x, train_y, val_y = train_test_split(X_train, y[train], test_size=0.2)
        dtrain = lgb.Dataset(train_x, label=train_y)
        dval = lgb.Dataset(val_x, label=val_y)
        
        params = {"objective": "regression",
                  "metric": "l2",
                  "verbose": -1,
                  "boosting_type": "gbdt",
                  "class_weight": "balanced",
                  "num_leaves": 31,
                  "max_depth": 10,
                  "min_child_samples": 20,
                  "feature_fraction": 1}

        model_rg = lgb.train(params, dtrain, valid_sets=[dtrain, dval], 
                             verbose_eval=100, early_stopping_rounds=50,
                             learning_rates = lambda iter: 0.05 * (0.99 ** iter))
        
        y_pred_rg = model_rg.predict(X_test, num_iteration=model_rg.best_iteration)
        
        # add model predictions for this drug to a dataframe
        scores.loc[ids[test], drug] = y_pred_rg

#### Ranking drugs for given cell lines

The top 10 drugs of the sorted array `pred` are the 10 most sentitive drugs for this given cell line based on model prediction. If we had a single train/val/test split rather than having to do cross-validation, we would have a single model and would be able to make predictions using the model for any cell line of interest that a user may input. But for this question I aggregated predictions for all cell lines in a dataframe for further analysis.

The ranked lists of drugs for each cell line were compared using both Kendall's Tau and Spearman’s rank correlation coefficients. These two correlation coefficients both assess statistical associations based on the ranks of the data, but Kendall's Tau is based on concordant and discordant pairs, whereas Spearman is based on deviations.

We can see from the results that the correlation between the model prediction and the experimental results varied quite a bit for different cell lines, some matching fairly well and the other not so much. The two metrics, Kendall's Tau and Spearman, were in agreement.

In [39]:
from scipy.stats import spearmanr, kendalltau

# dataframe for correlation scores between model prediction and experimental results
corr = pd.DataFrame(columns = ['Kendall_Tau','Kendall_Tau_top10drugs','Spearman','Spearman_top10drugs'], index = scores.index) 

for cell_line in scores.index:
    # sorting prediction and experimental scores for each given cell line
    pred = scores.loc[cell_line].sort_values()
    expt = exprmt.loc[cell_line].sort_values()
    
    # calculate Kendall Tau and Spearman correlations of all drugs from model prediction and experimental results
    corr.loc[cell_line, 'Kendall_Tau'] = kendalltau(pred.index.astype('str'),expt.index.astype('str'))[0]
    corr.loc[cell_line, 'Spearman'] = spearmanr(pred.index.astype('str'),expt.index.astype('str'))[0]
    # calculate Kendall Tau and Spearman correlations of just 10 top-ranking drugs from model prediction and experimental results
    corr.loc[cell_line, 'Kendall_Tau_top10drugs'] = kendalltau(pred.index[0:10].astype('str'),expt.index[0:10].astype('str'))[0]
    corr.loc[cell_line, 'Spearman_top10drugs'] = spearmanr(pred.index[0:10].astype('str'),expt.index[0:10].astype('str'))[0]

display(corr.head(10))

,Kendall_Tau,Kendall_Tau_top10drugs,Spearman,Spearman_top10drugs
GENE_SYMBOLS,,,,
906826,0.285992,0.422222,0.3976,0.539394
687983,-0.0385363,0.155556,-0.0553264,0.139394
910927,0.301887,0.2,0.41828,0.284848
1240138,0.24048,0.333333,0.334221,0.466667
1240139,0.408862,0.0222222,0.537224,-0.0424242
906792,0.396341,0.155556,0.505349,0.272727
910688,0.035506,0.422222,0.0512265,0.612121
1240135,0.614237,0.422222,0.761314,0.6
1290812,0.359005,-0.466667,0.474743,-0.539394
